In [8]:
import pandas as pd, numpy as np
import plotly.express as px

In [11]:
# read data
df = pd.read_csv('./data/women_in_parliament-historical_database-1945_to_2018_cleaned.csv')
df = df.query("chamber_type != 'upper' ")


In [12]:
# select data for 2018s
df_18 = df.query("year == 2018")
df_18 = df_18.dropna(subset=['women_perc'])

fig = px.treemap(df_18, path=[px.Constant("world"), 'region', 'country'], values='women_perc',
                  color='women_perc', title='Women Representation in Parliament 2018',
                  color_continuous_scale='RdBu',
                  color_continuous_midpoint=np.average(df_18['women_perc'], weights=df_18['women_perc']))
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [13]:
fig = px.bar(df_18, x='country', y='women_perc',
            labels=dict(country="Country", women_perc="Representation (%)"),
            title='Women Represetation in Parliament (2018) ')

# Add dropdown
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=["region","ASIA"],
                    label="ASIA",
                    method="update"
                ),
                dict(
                    args=["region", "EUR"],
                    label="EUR",
                    method="update"
                ),
                dict(
                    args=["region", "MENA"],
                    label="MENA",
                    method="update"
                ),
                dict(
                    args=["region", "PAC"],
                    label="PAC",
                    method="update"
                ),
                dict(
                    args=["region", "AME"],
                    label="AME",
                    method="update"
                )
            ]),
            direction="down",
            pad={"r": 2, "t": 2},
            showactive=True,
            x=0.0001,
            xanchor="center",
            y=1.2,
            yanchor="top"
        ),
    ]
)

fig.show()

In [28]:
df_18.head()

,country,region,election_renewal,year,month,chamber_type,chamber_total_seats,women_total,women_perc,notes,decade
12,Afghanistan,ASIA,Yes,2018,October,lower,244,68.0,0.28,NaN,2010
82,Antigua and Barbuda,AME,Yes,2018,March,lower,18,2.0,0.11,NaN,2010
140,Armenia,EUR,Yes,2018,December,single,132,32.0,0.24,NaN,2010
298,Barbados,AME,Yes,2018,May,lower,30,6.0,0.20,NaN,2010
399,Bhutan,ASIA,Yes,2018,October,lower,47,7.0,0.15,NaN,2010
